In [1]:
import pandas as pd
import numpy as np
train = pd.read_csv('/Users/gracegupta/Desktop/housing_data/train.csv')

In [2]:
train.head()

,Id,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,0,20,RL,NaN,9248,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,7,2009,WD,Normal,173000
1,1,70,RM,60.0,7200,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,10,2006,WD,Normal,157000
2,2,160,RM,24.0,1950,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,NaN,0,7,2008,COD,Normal,151000
3,3,20,RL,NaN,9790,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2009,WD,Normal,161500
4,4,50,RL,60.0,8064,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdWo,NaN,0,9,2008,WD,Normal,138000


In [3]:
train.shape

(2001, 81)

Get numeric data

In [92]:
train_numeric = train._get_numeric_data()

In [93]:
train_numeric.shape #there are 38 numeric columns including ID and SalePrice

(2001, 38)

Get categorical data

In [6]:
categorical_cols = [] #contains labels for all categorical columns
for col in train.columns:
    if col not in train_numeric.columns:
        categorical_cols.append(col)

In [7]:
train_categorical = train[categorical_cols]

In [8]:
train_categorical.shape #there are 43 categorical columns

(2001, 43)

# Dealing with categorical data

Look at distribution of null values

In [9]:
np.sum(train_categorical.isnull())

MS Zoning            0
Street               0
Alley             1872
Lot Shape            0
Land Contour         0
Utilities            0
Lot Config           0
Land Slope           0
Neighborhood         0
Condition 1          0
Condition 2          0
Bldg Type            0
House Style          0
Roof Style           0
Roof Matl            0
Exterior 1st         0
Exterior 2nd         0
Mas Vnr Type        16
Exter Qual           0
Exter Cond           0
Foundation           0
Bsmt Qual           53
Bsmt Cond           53
Bsmt Exposure       56
BsmtFin Type 1      53
BsmtFin Type 2      54
Heating              0
Heating QC           0
Central Air          0
Electrical           1
Kitchen Qual         0
Functional           0
Fireplace Qu       936
Garage Type        104
Garage Finish      106
Garage Qual        106
Garage Cond        106
Paved Drive          0
Pool QC           1989
Fence             1633
Misc Feature      1929
Sale Type            0
Sale Condition       0
dtype: int6

Drop columns with more than 1000 null values

In [10]:
train_cat_new = train_categorical.dropna(thresh=1000, axis=1)

In [11]:
np.sum(train_cat_new.isnull())

MS Zoning           0
Street              0
Lot Shape           0
Land Contour        0
Utilities           0
Lot Config          0
Land Slope          0
Neighborhood        0
Condition 1         0
Condition 2         0
Bldg Type           0
House Style         0
Roof Style          0
Roof Matl           0
Exterior 1st        0
Exterior 2nd        0
Mas Vnr Type       16
Exter Qual          0
Exter Cond          0
Foundation          0
Bsmt Qual          53
Bsmt Cond          53
Bsmt Exposure      56
BsmtFin Type 1     53
BsmtFin Type 2     54
Heating             0
Heating QC          0
Central Air         0
Electrical          1
Kitchen Qual        0
Functional          0
Fireplace Qu      936
Garage Type       104
Garage Finish     106
Garage Qual       106
Garage Cond       106
Paved Drive         0
Sale Type           0
Sale Condition      0
dtype: int64

In [12]:
train_cat_new.shape

(2001, 39)

# Using Cramer's V to identify multicollinearity among categorical variables

In [13]:
import scipy
from scipy import stats as ss

In [14]:
def cramers_v(x, y):
    confusion_matrix = pd.crosstab(x,y)
    chi2 = ss.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2/n
    r,k = confusion_matrix.shape
    phi2corr = max(0, phi2-((k-1)*(r-1))/(n-1))
    rcorr = r-((r-1)**2)/(n-1)
    kcorr = k-((k-1)**2)/(n-1)
    return np.sqrt(phi2corr/min((kcorr-1),(rcorr-1)))

In [15]:
for col in train_cat_new.columns: #col = x
    for col2 in train_cat_new.columns: #col2 = y
        if col2 != col:
            score = cramers_v(train_cat_new[col], train_cat_new[col2])
            if score > 0.5:
                print(col, col2) 
                print(score)

MS Zoning Utilities
0.998748591675947
MS Zoning Neighborhood
0.5234232681237401
MS Zoning Heating QC
0.5126112548776743
Utilities MS Zoning
0.998748591675947


/Users/gracegupta/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.


Neighborhood MS Zoning
0.5234232681237401
Neighborhood Exter Qual
0.5053663672565124
Exterior 1st Exterior 2nd
0.7754885964857037
Exterior 2nd Exterior 1st
0.7754885964857037
Exter Qual Neighborhood
0.5053663672565124
Exter Qual Kitchen Qual
0.5578627340476368
Heating QC MS Zoning
0.5126112548776743
Kitchen Qual Exter Qual
0.5578627340476368
Garage Qual Garage Cond
0.5237743296463221
Garage Cond Garage Qual
0.5237743296463221


# Fit model to features w/ low Cramer's V

In [16]:
col_to_exclude = ['MS Zoning', 'Exterior 2nd','Garage Cond', 'Sale Type']

In [17]:
X_train_cat = train_cat_new.drop(col_to_exclude, axis=1)

In [18]:
X_train_cat.shape

(2001, 35)

In [19]:
X_train_cat_dummies = pd.get_dummies(X_train_cat) #when creating dummies, all nulls get dropped

In [20]:
X_train_cat_dummies.shape

(2001, 210)

In [21]:
y = train_numeric["SalePrice"]

In [22]:
X_train_cat_dummies = X_train_cat_dummies.reindex(y.index) #reindex dummies

In [23]:
import statsmodels.api as sm

model_cat = sm.OLS(y, sm.add_constant(X_train_cat_dummies)).fit()

In [24]:
model_cat.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.851
Model:                            OLS   Adj. R-squared:                  0.836
Method:                 Least Squares   F-statistic:                     56.56
Date:                Sat, 22 Feb 2020   Prob (F-statistic):               0.00
Time:                        18:52:27   Log-Likelihood:                -23583.
No. Observations:                2001   AIC:                         4.753e+04
Df Residuals:                    1817   BIC:                         4.856e+04
Df Model:                         183                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                  -2.188e+16   7.57e+16     -0.289      0.772    -1.7e+17    1.27e+17
Street_Grvl             1.519e+16   4.27e+16      0.356      0.722   -6.86e+16     9.9e+16
Street_Pave             1.519e+16   4.27e+16      0.356      0.722   -6.86e+16     9.9e+16
Lot Shape_IR1          -3.958e+15   1.44e+16     -0.275      0.783   -3.21e+16    2.42e+16
Lot Shape_IR2          -3.958e+15   1.44e+16     -0.275      0.783   -3.21e+16    2.42e+16
Lot Shape_IR3          -3.958e+15   1.44e+16     -0.275      0.783   -3.21e+16    2.42e+16
Lot Shape_Reg          -3.958e+15   1.44e+16     -0.275      0.783   -3.21e+16    2.42e+16
Land Contour_Bnk        4.961e+15    1.4e+16      0.355      0.722   -2.24e+16    3.23e+16
Land Contour_HLS        4.961e+15    1.4e+16      0.355      0.722   -2.24e+16    3.23e+16
Land Contour_Low        4.961e+15    1.4e+16      0.355      0.722   -2.24e+16    3.23e+16
Land Contour_Lvl        4.961e+15    1.4e+16      0.355      0.722   -2.24e+16    3.23e+16
Utilities_AllPub        1.443e+16   4.06e+16      0.356      0.722   -6.51e+16     9.4e+16
Utilities_NoSewr        1.443e+16   4.06e+16      0.356      0.722   -6.51e+16     9.4e+16
Lot Config_Corner       1.632e+14   3.94e+15      0.041      0.967   -7.56e+15    7.88e+15
Lot Config_CulDSac      1.632e+14   3.94e+15      0.041      0.967   -7.56e+15    7.88e+15
Lot Config_FR2          1.632e+14   3.94e+15      0.041      0.967   -7.56e+15    7.88e+15
Lot Config_FR3          1.632e+14   3.94e+15      0.041      0.967   -7.56e+15    7.88e+15
Lot Config_Inside       1.632e+14   3.94e+15      0.041      0.967   -7.56e+15    7.88e+15
Land Slope_Gtl           8.06e+15   2.26e+16      0.356      0.722   -3.63e+16    5.24e+16
Land Slope_Mod           8.06e+15   2.26e+16      0.356      0.722   -3.63e+16    5.24e+16
Land Slope_Sev           8.06e+15   2.26e+16      0.356      0.722   -3.63e+16    5.24e+16
Neighborhood_Blmngtn   -2.652e+14   7.87e+14     -0.337      0.736   -1.81e+15    1.28e+15
Neighborhood_Blueste   -2.652e+14   7.87e+14     -0.337      0.736   -1.81e+15    1.28e+15
Neighborhood_BrDale    -2.652e+14   7.87e+14     -0.337      0.736   -1.81e+15    1.28e+15
Neighborhood_BrkSide   -2.652e+14   7.87e+14     -0.337      0.736   -1.81e+15    1.28e+15
Neighborhood_ClearCr   -2.652e+14   7.87e+14     -0.337      0.736   -1.81e+15    1.28e+15
Neighborhood_CollgCr   -2.652e+14   7.87e+14     -0.337      0.736   -1.81e+15    1.28e+15
Neighborhood_Crawfor   -2.652e+14   7.87e+14     -0.337      0.736   -1.81e+15    1.28e+15
Neighborhood_Edwards   -2.652e+14   7.87e+14     -0.337      0.736   -1.81e+15    1.28e+15
Neighborhood_Gilbert   -2.652e+14   7.87e+14     -0.337      0.736   -1.81e+15    1.28e+15
Neighborhood_Greens    -2.652e+14   7.87e+14     -0.337      0.736   -1.81e+15  

# Dealing with numeric data

Look at distribution of nulls

In [94]:
np.sum(train_numeric.isnull())

Id                   0
MS SubClass          0
Lot Frontage       332
Lot Area             0
Overall Qual         0
Overall Cond         0
Year Built           0
Year Remod/Add       0
Mas Vnr Area        16
BsmtFin SF 1         1
BsmtFin SF 2         1
Bsmt Unf SF          1
Total Bsmt SF        1
1st Flr SF           0
2nd Flr SF           0
Low Qual Fin SF      0
Gr Liv Area          0
Bsmt Full Bath       1
Bsmt Half Bath       1
Full Bath            0
Half Bath            0
Bedroom AbvGr        0
Kitchen AbvGr        0
TotRms AbvGrd        0
Fireplaces           0
Garage Yr Blt      106
Garage Cars          1
Garage Area          1
Wood Deck SF         0
Open Porch SF        0
Enclosed Porch       0
3Ssn Porch           0
Screen Porch         0
Pool Area            0
Misc Val             0
Mo Sold              0
Yr Sold              0
SalePrice            0
dtype: int64

In [95]:
def preprocessing_pipeline(train):
    ## Your code goes here
    
    #fill NAs with median
    train = train.fillna(train.median())

    #get rid of outliers
    for X in train:
        mu = np.mean(train[X])
        std = np.std(train[X])
        normalized_data = (train[X]-mu)/std
        indexes = normalized_data < 3
        processed_train = train.loc[indexes, :]
    
    return processed_train

Preprocess data

In [96]:
train_numeric = preprocessing_pipeline(train_numeric)

In [97]:
train_numeric.shape

(1969, 38)

Drop the "Id" column.

In [98]:
train_numeric = train_numeric.drop(columns=['Id'])

# Calculate VIFs for numeric data


In [31]:
def calculate_vif(r_squared):
    ## Your code goes here
    vif = 1/(1-r_squared)
    return vif

In [32]:
import statsmodels.api as sm

def generate_vif_dataframe(processed_train):
    ## Your code goes here
    data = []
    for X in processed_train:
        X_list = []
        X_list.append(X)
        i = processed_train.columns.get_loc(X)
        reg_list = [] #contains all other variables for regression against X
        for j in range(len(processed_train.columns)):
            if j != i :
                reg_list.append(processed_train.columns[j])
        model = sm.OLS(processed_train[X], sm.add_constant(processed_train[reg_list])).fit()
        rsq = model.rsquared
        vif = calculate_vif(rsq)
        X_list.append(vif)
        data.append(X_list)
        
    vif_dataframe = pd.DataFrame(data, columns = ['Variable name','VIF'])
                                
    return vif_dataframe

Select all features

In [99]:
numeric_features = train_numeric.loc[:, train_numeric.columns != 'SalePrice']

In [100]:
vif_dataframe = generate_vif_dataframe(numeric_features)
vif_dataframe.sort_values('VIF',ascending=False)

/Users/gracegupta/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until


,Variable name,VIF
14,Low Qual Fin SF,inf
15,Gr Liv Area,inf
12,1st Flr SF,inf
13,2nd Flr SF,inf
8,BsmtFin SF 1,1.440326e+04
10,Bsmt Unf SF,1.412280e+04
11,Total Bsmt SF,1.341112e+04
9,BsmtFin SF 2,2.005389e+03
25,Garage Cars,5.729113e+00
26,Garage Area,5.452137e+00


# Building model based on VIFs

In [101]:
columns_to_exclude = ['BsmtFin SF 1','Garage Cars','Year Built','Total Bsmt SF','TotRms AbvGrd','Garage Yr Blt','Full Bath','Low Qual Fin SF','Gr Liv Area','1st Flr SF',
                     'Low Qual Fin SF','Gr Liv Area','1st Flr SF','2nd Flr SF','Year Remod/Add','Garage Area']
features_updated = numeric_features.drop(columns_to_exclude, axis=1)
vifs = generate_vif_dataframe(features_updated)
vifs.sort_values('VIF',ascending=False)

,Variable name,VIF
3,Overall Qual,1.722018
7,Bsmt Unf SF,1.665562
1,Lot Frontage,1.522738
8,Bsmt Full Bath,1.515370
0,MS SubClass,1.454611
13,Fireplaces,1.346139
11,Bedroom AbvGr,1.310538
12,Kitchen AbvGr,1.292194
10,Half Bath,1.291483
2,Lot Area,1.265542


In [102]:
y = train_numeric["SalePrice"]

In [103]:
features_updated.shape

(1969, 23)

In [104]:
y.shape

(1969,)

In [85]:
#features_updated = features_updated.reindex(y_test.index)

Fit model to chosen numeric features

In [106]:
model_numeric = sm.OLS(y, sm.add_constant(features_updated)).fit()

In [107]:
model_numeric.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.755
Model:                            OLS   Adj. R-squared:                  0.752
Method:                 Least Squares   F-statistic:                     260.8
Date:                Sat, 22 Feb 2020   Prob (F-statistic):               0.00
Time:                        19:24:49   Log-Likelihood:                -23359.
No. Observations:                1969   AIC:                         4.677e+04
Df Residuals:                    1945   BIC:                         4.690e+04
Df Model:                          23                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const           2.676e+06   1.21e+06      2.204      0.028    2.95e+05    5.06e+06
MS SubClass     -108.1830     22.440     -4.821      0.000    -152.192     -64.174
Lot Frontage     233.5893     46.388      5.036      0.000     142.613     324.566
Lot Area           0.6172      0.120      5.156      0.000       0.382       0.852
Overall Qual    3.118e+04    743.830     41.913      0.000    2.97e+04    3.26e+04
Overall Cond     -43.1749    743.093     -0.058      0.954   -1500.517    1414.167
Mas Vnr Area      42.3912      5.317      7.973      0.000      31.963      52.819
BsmtFin SF 2      -4.1096      4.896     -0.839      0.401     -13.711       5.491
Bsmt Unf SF        6.7774      2.266      2.990      0.003       2.333      11.222
Bsmt Full Bath  1.862e+04   1820.673     10.228      0.000    1.51e+04    2.22e+04
Bsmt Half Bath  3241.7982   3442.633      0.942      0.346   -3509.840    9993.436
Half Bath       8887.5724   1757.641      5.057      0.000    5440.515    1.23e+04
Bedroom AbvGr   6437.2789   1061.792      6.063      0.000    4354.909    8519.648
Kitchen AbvGr   -709.2808   4328.839     -0.164      0.870   -9198.933    7780.371
Fireplaces      1.214e+04   1415.100      8.576      0.000    9359.997    1.49e+04
Wood Deck SF      50.9827      6.707      7.602      0.000      37.830      64.136
Open Porch SF     40.0707     12.353      3.244      0.001      15.844      64.297
Enclosed Porch    -8.9727     12.859     -0.698      0.485     -34.192      16.247
3Ssn Porch        46.0909     28.900      1.595      0.111     -10.588     102.770
Screen Porch      36.8044     14.586      2.523      0.012       8.199      65.410
Pool Area        -55.7224     20.173     -2.762      0.006     -95.285     -16.159
Misc Val          -6.3447      1.299     -4.883      0.000      -8.893      -3.797
Mo Sold          -97.4375    291.479     -0.334      0.738    -669.081     474.206
Yr Sold        -1371.7113    604.664     -2.269      0.023   -2557.569    -185.854
==============================================================================
Omnibus:                      254.348   Durbin-Watson:                   2.045
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2903.622
Skew:                           0.002   Prob(JB):                         0.00
Kurtosis:                       8.949   Cond. No.                     1.97e+07
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.97e+07. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

# Make model from both categorical and continuous data.

In [108]:
X_train = pd.concat([X_train_cat_dummies, features_updated],axis=1)

In [110]:
X_train = X_train.dropna(axis=0)

In [112]:
X_train.shape

(1969, 233)

In [113]:
y = train_numeric["SalePrice"]

In [114]:
y.shape

(1969,)

In [115]:
model_both = sm.OLS(y, sm.add_constant(X_train)).fit()

In [116]:
model_both.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.893
Model:                            OLS   Adj. R-squared:                  0.881
Method:                 Least Squares   F-statistic:                     71.55
Date:                Sat, 22 Feb 2020   Prob (F-statistic):               0.00
Time:                        19:30:33   Log-Likelihood:                -22542.
No. Observations:                1969   AIC:                         4.550e+04
Df Residuals:                    1762   BIC:                         4.665e+04
Df Model:                         206                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                   3.334e+05   1.43e+05      2.332      0.020     5.3e+04    6.14e+05
Street_Grvl             1.469e+05   7.17e+04      2.048      0.041    6216.553    2.88e+05
Street_Pave             1.865e+05   7.16e+04      2.603      0.009     4.6e+04    3.27e+05
Lot Shape_IR1           8.443e+04   3.58e+04      2.356      0.019    1.41e+04    1.55e+05
Lot Shape_IR2            8.67e+04   3.59e+04      2.414      0.016    1.63e+04    1.57e+05
Lot Shape_IR3           7.835e+04   3.64e+04      2.154      0.031    7022.192     1.5e+05
Lot Shape_Reg            8.39e+04   3.59e+04      2.339      0.019    1.36e+04    1.54e+05
Land Contour_Bnk        7.585e+04   3.59e+04      2.114      0.035    5477.293    1.46e+05
Land Contour_HLS        9.187e+04   3.59e+04      2.561      0.011    2.15e+04    1.62e+05
Land Contour_Low        8.052e+04   3.58e+04      2.249      0.025    1.03e+04    1.51e+05
Land Contour_Lvl        8.512e+04   3.59e+04      2.374      0.018    1.48e+04    1.55e+05
Utilities_AllPub        1.489e+05   7.27e+04      2.049      0.041    6351.605    2.91e+05
Utilities_NoSewr        1.845e+05   7.29e+04      2.530      0.011    4.15e+04    3.28e+05
Lot Config_Corner       6.792e+04   2.87e+04      2.368      0.018    1.17e+04    1.24e+05
Lot Config_CulDSac      7.654e+04   2.88e+04      2.662      0.008    2.01e+04    1.33e+05
Lot Config_FR2          6.006e+04   2.89e+04      2.078      0.038    3375.485    1.17e+05
Lot Config_FR3          5.994e+04   2.91e+04      2.058      0.040    2811.216    1.17e+05
Lot Config_Inside       6.892e+04   2.87e+04      2.403      0.016    1.27e+04    1.25e+05
Land Slope_Gtl          1.172e+05   4.78e+04      2.449      0.014    2.34e+04    2.11e+05
Land Slope_Mod          1.223e+05   4.79e+04      2.555      0.011    2.84e+04    2.16e+05
Land Slope_Sev          9.391e+04   4.84e+04      1.941      0.052    -970.696    1.89e+05
Neighborhood_Blmngtn    1.389e+04   8138.821      1.706      0.088   -2076.958    2.98e+04
Neighborhood_Blueste     556.8548   1.17e+04      0.048      0.962   -2.23e+04    2.34e+04
Neighborhood_BrDale     -834.7838   8956.793     -0.093      0.926   -1.84e+04    1.67e+04
Neighborhood_BrkSide   -3192.8072   6543.857     -0.488      0.626    -1.6e+04    9641.732
Neighborhood_ClearCr    4266.6087   7285.014      0.586      0.558      -1e+04    1.86e+04
Neighborhood_CollgCr    -182.7670   5804.373     -0.031      0.975   -1.16e+04    1.12e+04
Neighborhood_Crawfor    1.963e+04   6206.069      3.163      0.002    7457.146    3.18e+04
Neighborhood_Edwards   -1.067e+04   5890.258     -1.812      0.070   -2.22e+04     882.246
Neighborhood_Gilbert   -7678.7027   6192.737     -1.240      0.215   -1.98e+04    4467.183
Neighborhood_Greens     5579.5487   1.14e+04      0.491      0.623   -1.67e+04  

# Predict on test data

In [117]:
#Read the test set in
test = pd.read_csv('/Users/gracegupta/Desktop/housing_data/test.csv')

In [118]:
test.shape

(929, 80)

In [119]:
test_numeric = test._get_numeric_data()

In [120]:
test_numeric.shape

(929, 37)

Preprocess data

In [121]:
test_numeric = preprocessing_pipeline(test_numeric)

In [122]:
test_numeric.shape

(929, 37)

Drop the "Id" column.

In [123]:
test_numeric = test_numeric.drop(columns=['Id'])

In [124]:
test_numeric = test_numeric[features_updated.columns]

In [125]:
test_numeric.shape

(929, 23)

Get categorical test data

Use dummy variables

In [152]:
categorical_cols = [] #contains labels for all categorical columns
for col in test.columns:
    if col not in test_numeric.columns:
        categorical_cols.append(col)

In [155]:
test_categorical = test[train_cat_new.columns]

In [156]:
test_categorical.shape

(929, 39)

In [157]:
test_categorical_dummies = pd.get_dummies(test_categorical)

In [158]:
missing_cols = set( X_train_cat_dummies.columns ) - set( test_categorical_dummies.columns )

In [159]:
missing_cols

{'Condition 2_RRAe',
 'Condition 2_RRAn',
 'Electrical_Mix',
 'Exterior 1st_BrkComm',
 'Exterior 1st_ImStucc',
 'Exterior 1st_PreCast',
 'Exterior 1st_Stone',
 'Functional_Sal',
 'Heating_Floor',
 'Kitchen Qual_Po',
 'Mas Vnr Type_CBlock',
 'Neighborhood_Landmrk',
 'Roof Matl_ClyTile',
 'Roof Matl_Membran',
 'Roof Matl_Roll'}

In [160]:
for c in missing_cols:
    test_categorical_dummies[c] = 0

In [161]:
test_categorical_dummies = test_categorical_dummies[X_train_cat_dummies.columns]

In [162]:
test_categorical_dummies.shape

(929, 210)

In [163]:
X_train_cat_dummies.shape

(2001, 210)

In [164]:
X_test = pd.concat([test_categorical_dummies, test_numeric], axis=1)

In [165]:
X_test.shape

(929, 233)

In [166]:
pred = model_both.predict(sm.add_constant(X_test))

In [172]:
sample_submission = pd.read_csv('/Users/gracegupta/Desktop/housing_data/sample_submission (1).csv')
sample_submission.loc[:, 'SalePrice'] = pred
sample_submission.to_csv('hw3_attemp1.csv', header=True, index=False)
sample_submission.head()

,Id,SalePrice
0,0,104349.836407
1,1,237790.178755
2,2,112407.221944
3,3,145092.553180
4,4,172581.193298
